# Read From FeatureStore Demo
documentation: https://sagemaker.readthedocs.io/en/stable/amazon_sagemaker_featurestore.html

In [ ]:
import boto3
import sagemaker
import json
from datetime import datetime, timedelta
from sagemaker.session import Session

In [ ]:
sagemaker_session = sagemaker.Session()
# role = sagemaker.get_execution_role()
# There is a bug, see: https://github.com/aws/sagemaker-python-sdk/issues/4419
# temporary workaround:

NOTEBOOK_METADATA_FILE = "/opt/ml/metadata/resource-metadata.json"
with open(NOTEBOOK_METADATA_FILE, "rb") as f:
    metadata = json.loads(f.read())
    instance_name = metadata["ResourceName"]
    domain_id = metadata.get("DomainId")
    user_profile_name = metadata.get("UserProfileName")
    space_name = metadata.get("SpaceName")
domain_desc = sagemaker_session.sagemaker_client.describe_domain(DomainId=domain_id)
if "DefaultSpaceSettings" in domain_desc:
    role = domain_desc["DefaultSpaceSettings"]["ExecutionRole"]
else:
    role = domain_desc["DefaultUserSettings"]["ExecutionRole"]
# end of bug workaround

region = sagemaker_session.boto_region_name

boto_session = boto3.Session(region_name=region)

default_bucket = sagemaker_session.default_bucket()
prefix = "sagemaker-featurestore"

In [ ]:
ssm_client = boto3.client("ssm")
# Read the Feature Group Name from SSM Parameters
feature_group_name = ssm_client.get_parameter(
    Name="/rdi-mlops/stack-parameters/sagemaker-feature-group-name"
)["Parameter"]["Value"]

In [ ]:
sagemaker_client = boto_session.client(service_name="sagemaker", region_name=region)
featurestore_runtime = boto_session.client(
    service_name="sagemaker-featurestore-runtime", region_name=region
)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime,
)

In [ ]:
fg_list = sagemaker_client.list_feature_groups()
fg_list

In [ ]:
latest_record_time = datetime.utcnow() - timedelta(hours=0, minutes=5)
latest_record_time = latest_record_time.replace(second=0, microsecond=0)
response = featurestore_runtime.get_record(
    FeatureGroupName=feature_group_name,
    RecordIdentifierValueAsString=latest_record_time.strftime("%Y-%m-%d %H:%M:%S.%f")[
        :-7
    ],
)
response